In [2]:
import pandas as pd

In [ ]:
def extract_write_image_lines(file_path):
    """
    Reads a text file and extracts all lines starting with 'WRITE_IMAGE',
    splits them into space-separated values, and returns a DataFrame.
    
    Args:
        file_path (str): Path to the input text file.
    
    Returns:
        pd.DataFrame: DataFrame containing the split data from lines.
    """
    extracted_data = []
    
    with open(file_path, 'r') as file:
        for line in file:
            stripped = line.strip()
            if stripped.startswith('WRITE_IMAGE') and 'UP' not in stripped:
                parts = stripped.split()
                extracted_data.append(parts)
    
    # Optional: create column names like col0, col1, col2... based on max number of parts
    if extracted_data:
        max_len = max(len(row) for row in extracted_data)
        column_names = [f'col{i}' for i in range(max_len)]
    else:
        column_names = []
    
    df = pd.DataFrame(extracted_data, columns=column_names)
    return df

test_file = "/Users/nicholasbachand/Documents/Research/Cascade/city_block_cfd/CHARLES/config2/R60/charles.in"
image_df = extract_write_image_lines(test_file)


In [28]:
image_df.drop(columns=[f'col{i}' for i in range(len(image_df.columns)) if i not in [4,6,12,13,15,19]], inplace=True)  # Drop the first two columns
image_df.columns = ["x", "z", "x_pixels", "z_pixels", "width", "y"]
image_df

,x,z,x_pixels,z_pixels,width,y
0,0,0,480,480,232,1.5
1,0,0,480,480,232,3
2,0,0,480,480,232,12
3,0,0,480,480,232,1.5
4,0,0,480,480,232,1.5
...,...,...,...,...,...,...
161,-60.0,-12.0,480,480,24,1.5
162,-52.0,-36.0,480,480,24,1.5
163,-60.0,-60.0,480,480,24,1.5
164,-52.0,-84.0,480,480,24,1.5


In [60]:
multiRun_dir = "CHARLES/multiRuns"
pcStatsMI = pd.read_csv(f"{multiRun_dir}/pointCloudStatsNoIntMI.csv", index_col=[0,1])
pcStatsMI.columns = pcStatsMI.columns.str.replace("-noInt", "")
pcStatsMI = pcStatsMI.droplevel(0)
pcStatsMI = pcStatsMI.drop_duplicates()
pcStatsMI = pcStatsMI.reset_index()
pcStatsMI = pcStatsMI.set_index(["run", "index"])

pcStatsMI

comp(u_avg,0)  comp(u_avg,1)  comp(u_avg,2)  \
run    index                                                                  
2570.0 skylight_0-0_h_-1-0_B        -0.656469      -0.306277      -0.438879   
       skylight_0-0_h_-1-0_Bx        0.337597      -0.249700       0.227412   
       skylight_0-0_h_-1-0_Bxz      -0.349963      -0.176104       0.255243   
       skylight_0-0_h_-1-0_Bz       -0.359206      -0.243828      -1.204053   
       skylight_0-0_h_0--1_B        -0.422069      -0.276880      -0.392550   
...                                       ...            ...            ...   
3282.0 zwindow_2-1_h_0-1_Bz          0.037347      -0.386544       1.052185   
       zwindow_2-1_h_0-2_B          -1.757011      -0.027264       0.028324   
       zwindow_2-1_h_0-2_Bx         -0.042986       0.956992      -0.337208   
       zwindow_2-1_h_0-2_Bxz        -0.415027       0.028800      -0.064438   
       zwindow_2-1_h_0-2_Bz          0.024170      -0.332080       1.042243   

                                   p_avg     p_rms    windowType openingType  \
run    index                                                                   
2570.0 skylight_0-0_h_-1-0_B    0.313373  0.367579  skylight_0-0    skylight   
       skylight_0-0_h_-1-0_Bx  -1.098614  0.665496  skylight_0-0    skylight   
       skylight_0-0_h_-1-0_Bxz -0.193095  0.230426  skylight_0-0    skylight   
       skylight_0-0_h_-1-0_Bz   0.016770  0.272428  skylight_0-0    skylight   
       skylight_0-0_h_0--1_B    0.135680  0.259910  skylight_0-0    skylight   
...                                  ...       ...           ...         ...   
3282.0 zwindow_2-1_h_0-1_Bz    -3.014041  1.853741   zwindow_2-1     zwindow   
       zwindow_2-1_h_0-2_B      0.004176  0.392323   zwindow_2-1     zwindow   
       zwindow_2-1_h_0-2_Bx     3.622253  2.021010   zwindow_2-1     zwindow   
       zwindow_2-1_h_0-2_Bxz   -0.850557  1.078640   zwindow_2-1     zwindow   
       zwindow_2-1_h_0-2_Bz    -2.854836  1.446887   zwindow_2-1     zwindow   

                               windowNumber houseType blockType  ...  \
run    index                                                     ...   
2570.0 skylight_0-0_h_-1-0_B            0-0      -1-0         B  ...   
       skylight_0-0_h_-1-0_Bx           0-0      -1-0        Bx  ...   
       skylight_0-0_h_-1-0_Bxz          0-0      -1-0       Bxz  ...   
       skylight_0-0_h_-1-0_Bz           0-0      -1-0        Bz  ...   
       skylight_0-0_h_0--1_B            0-0      0--1         B  ...   
...                                     ...       ...       ...  ...   
3282.0 zwindow_2-1_h_0-1_Bz             2-1       0-1        Bz  ...   
       zwindow_2-1_h_0-2_B              2-1       0-2         B  ...   
       zwindow_2-1_h_0-2_Bx             2-1       0-2        Bx  ...   
       zwindow_2-1_h_0-2_Bxz            2-1       0-2       Bxz  ...   
       zwindow_2-1_h_0-2_Bz             2-1       0-2        Bz  ...   

                                EP_comp(u_avg,1)  EP_comp(u_avg,2)  \
run    index                                                         
2570.0 skylight_0-0_h_-1-0_B            0.307974          0.533049   
       skylight_0-0_h_-1-0_Bx           0.105307          0.077018   
       skylight_0-0_h_-1-0_Bxz          0.165404         -0.250992   
       skylight_0-0_h_-1-0_Bz           0.262501          1.295020   
       skylight_0-0_h_0--1_B            0.269689          0.421002   
...                                          ...               ...   
3282.0 zwindow_2-1_h_0-1_Bz             0.317686         -0.857391   
       zwindow_2-1_h_0-2_B             -0.077122         -0.066760   
       zwindow_2-1_h_0-2_Bx            -0.827463          0.452609   
       zwindow_2-1_h_0-2_Bxz           -0.272383          0.158857   
       zwindow_2-1_h_0-2_Bz             0.298914         -0.873819   

                                EP_mag(u)_avg  EP_p_avg    EP_mag  \
run    index                                  